In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
from typing import Tuple

from roveranalyzer.utils import Project
import contextily as ctx
from roveranalyzer.analysis import OppAnalysis, DensityMap
import roveranalyzer.simulators.crownet.dcd as Dmap
import roveranalyzer.simulators.opp as OMNeT

def run_data(run, sim, hdf_file="data.h5") -> Tuple[Dmap.DcdHdfBuilder, OMNeT.CrownetSql]:
    data_root = f"{os.environ['HOME']}/repos/crownet/crownet/simulations/{sim}/analysis.d/results/{run}"
    builder = Dmap.DcdHdfBuilder.get(hdf_file, data_root).epsg(Project.UTM_32N)
    
    sql = OMNeT.CrownetSql(
            vec_path=f"{data_root}/vars_rep_0.vec", 
            sca_path=f"{data_root}/vars_rep_0.sca", 
        network="World")
    return data_root, builder, sql

beacon_app = ".app[1].app"
density_map = ".app[0].app"

In [ ]:

# root, b1, sql1 = run_data("vadere_XX_20211206-17:12:27", "mucFreiheitLte")
# root, b1, sql1 = run_data("subwayStationary_multiEnb_20211213-13:05:44", "mucFreiheitLte")
root, b1, sql1 = run_data("subwayStationary_multiEnb_20211213-17:34:54", "mucFreiheitLte")

p1 = b1.build(override_hdf=False)
dcd = b1.build_dcdMap()

In [ ]:
fig, ax = dcd.plot_count_diff(savefig=f"{root}/out/count_over_time.pdf")

In [ ]:
fig, ax = dcd.plot_error_quantil_histogram(
    stat="percent",
    savefig=f"{root}/out/error_histogram_quantil.pdf"
    )
fig, ax = dcd.plot_error_histogram(
    stat="percent",
    savefig=f"{root}/out/error_histogram.pdf"
)

In [ ]:
data, idx = OppAnalysis.get_neighborhood_table_size(sql=sql1)
fig, ax = OppAnalysis.plot_neighborhood_table_size_over_time(
    data, idx, savefig=f"{root}/out/neighborhood_table_size.pdf"
)

In [ ]:
beacon_df = OppAnalysis.get_packet_source_distribution(
    sql1, app_path=".app[1].app")
fig, ax = OppAnalysis.plot_packet_source_distribution(beacon_df)
ax.set_title("Beacon packet source distribution")
fig.savefig(f"{root}/out/beacon_pkt_source_dis.pdf")

In [ ]:
m = sql1.module_path(".app[0].app", "pNode")
df_delay = sql1.vec_data(
    module_name="World.pNode[%].app[0].app",
    vector_name="rcvdPkLifetime:vector", 
    add_host_ids=True
)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,9))

ax = sns.histplot(
    age_dcd["packet_age"], 
    stat="percent", fill=False, legend=True, ax=axes[0], 
)
ax.set_title("Density map packet delay")
ax.set_xlim([0.010, 0.030])
ax =sns.histplot(
    age_beacon["packet_age"], cumulative=True,
    stat="percent", element="step", fill=False, legend=True, ax=axes[1], 
)
ax.set_title("Beacon Packet Delay")
ax.set_xlim([0.010, 0.030])
fig.tight_layout()

welches Verfahren Bewerten die Performance 


Mittleren Fehler über alle 

* Statisch 
  * ymf
  * ymf+
  * 
* Beacon Interval 
* Dichtekarten Interval
* 